In [67]:
using LinearAlgebra,Serialization

### Functions 

Some basics of my setup of exact diagonalization and the functions are explained below. 
We simply use the computational basis as our basis for the matrix representation. 
* A "basis" (computational basis) is an array storing the qubit configuration. e.g. [0,0,0,0] or [1,0,1,0] etc. 
* A "code" is a number which encodes the basis, which is encoded as $ \sum_{i=1}^{L}basis[i]*d^{i-1}$. (For qubit, $d=2$.)  

**codetobasis** and **basistocode** convert code to basis and basis to code, respectively.

Note that since Julia labels an array starting with 1, the indices of a vector and the codes are converted as $index=code+1$. Here the conversion is simple since we do not use symmetry or conservation laws of the system. So all the codes $a=0 \dots 2^N-1$ are used. In practice, if we need to use symmtry or conservation law, I would prepare a dictionary *Dict()* to record which index corresponds to which code.

> To form a Hamiltonian matrix or a Linbladian matrix, it is easier to have a function which takes an operator *$\hat{O}$* operating on *sites* and a basis state *$|code\rangle$*, outputing $\hat{O}_{sites}|code\rangle$.

The function **opcode** takes a matrix *op*, which operates on the sites *sites* on a *code* basis state with *N* sites and total Hilbert space dimension *dim*.
For example, if one would like to operate $\hat{O}=X_1Y_4$ on $|5\rangle$, then 
~~~
σx=[0 1;1 0]
σy=[0 -1im;1im 0]
σz=[1 0;0 -1];

op=kron(σy,σz) #Note the reverse order of kron in julia
code=5;
N=6;
dim=2^N;

outvec=opcode(op,[1,4],code,N,dim)
~~~


In [68]:
#convert a state code "a" to computational basis, e.g. 1=[1,0,0,0,0]
function codetobasis(a::Int64,N::Int64,d=2) #a: code, N:number of quditsm d:local Hilbertspace dimension
    basis=zeros(Int64,N);
    temp=a;
    for i=1:N
        basis[i]=rem(temp,d);
        temp=div(temp,d);
    end
    return basis;    
end

#basis to code converter
function basistocode(basis::Array{Int64},d=2)
    temp=0;
    for i=1:length(basis)
        temp+=basis[i]*d^(i-1);
    end    
    return temp;  
end

function opcode(op,sites,code,N,dim) # |output⟩ = Op_{sites} |code⟩; |code⟩ is a computational-basis state
    outvec=zeros(eltype(op),dim)
    basis=codetobasis(code,N)
    ind=basistocode(basis[sites])+1
    
    for i=1:size(op)[1]
        newbasis=copy(basis)
        nbs=codetobasis(i-1,length(sites))

        newbasis[sites]=nbs
        newindex=basistocode(newbasis)+1
        outvec[newindex] += op[i,ind]
    end
    return outvec
end

opcode (generic function with 1 method)

### Matrix representation of a Linbladian
It is convenient to "flip" the bra of the density matrices into ket ("vectorize" it). 

$\rho =\sum_{a,b=0}^{2^N-1} \rho_{ab}|a\rangle \langle b| \leftrightarrow |\rho\rangle\rangle =\sum_{a,b=0}^{2^N-1} \rho_{ab}|a\rangle |b\rangle$.

We will then use a basis with $2N$ sites to denote the computational basis for the Linbladian. The first $N$ denotes the configuration from code a and the last $N$ denotes the configuration from code b. In this vectorized representation, a Linbladian is written as 

$\mathbf{L} = \sum_{i} L_i \otimes L^*_i - \frac{1}{2} L_i^{\dagger}L_i \otimes I - \frac{1}{2} I \otimes (L_i^{\dagger}L_i)^{T} $.

* The function *JLJRcode* realizes $L_i \otimes L^*_i$.
* The function *JLJLcode* realizes $L_i^{\dagger}L_i \otimes I$.
* The function *JRJRcode* realizes $I \otimes (L_i^{\dagger}L_i)^{T}$.

> Note that in this way of organizing the basis, one can convert $\rho$ to $|\rho\rangle$ in Julia using *rhovec=reshape(rho,dim^2)* and $|\rho\rangle$ to $\rho$ using *rho=reshape(rhovec,dim,dim)*

In [69]:
function JLJRcode(Jumpop,sites,code,N,dim) # JL ⊗ JR^{*} |code⟩
    op=kron(conj(Jumpop),Jumpop)
    return opcode(op,[sites;sites .+ N],code,2N,dim^2)
end

function JLJLcode(Jumpop,sites,code,N,dim) # JL^{†}* JL |code⟩
    op=Jumpop'*Jumpop
    return opcode(op,sites,code,2N,dim^2)
end

function JRJRcode(Jumpop,sites,code,N,dim) # transpose(JR^{†}* JR) |code⟩
    op=transpose(Jumpop'*Jumpop)
    return opcode(op,sites .+N,code,2N,dim^2)
end

Linbcode(Jumpop,sites,code,N,dim)=JLJRcode(Jumpop,sites,code,N,dim) - 0.5*JLJLcode(Jumpop,sites,code,N,dim)- 0.5*JRJRcode(Jumpop,sites,code,N,dim)


Linbcode (generic function with 1 method)

Construct $\mathbf{Z}_2$ symmetry operators 

In [70]:
function getZ2e(L)
    op=1
    Id=Matrix(I,2,2)
    for j=1:L
       rem(j,2) == 0 ? op=kron(σx,op) :  op=kron(Id,op) 
    end
    return op
end

function getZ2o(L)
    op=1
    Id=Matrix(I,2,2)
    for j=1:L
       rem(j,2) == 1 ? op=kron(σx,op) :  op=kron(Id,op) 
    end
    return op
end

getZ2o (generic function with 1 method)

### Forming the $ρ_{cluster}$ and $\Psi_z$ configurations for later use  
The fixed point mixed state is
$\rho_{clus} = \frac{1}{2^{N/2}}\sum_{ z_{2n+1}} |\Psi_{z_{2n+1}} \rangle \langle \Psi_{z_{2n+1}} |$

In [186]:
L=4
dim=2^L
Linbdim=dim^2

σx=[0 1;1 0]
σy=[0 -1im;1im 0]
σz=[1 0;0 -1];

xp=[1,1]/sqrt(2)    #|+⟩
xm=[1,-1]/sqrt(2)   #|-⟩
z0=[1,0]            #|0⟩
z1=[0,-1];          #|1⟩

In [187]:
#Symmetry operators
U2e=getZ2e(L)
U2o=getZ2o(L);
Id=Matrix(I,dim,dim)
Z2ee=kron(U2e,U2e)
Z2oo=kron(U2o,U2o);
Z2eI=kron(U2e,Id)
Z2Ie=kron(Id,U2e);

In [188]:
function getΨzstate(zconfig) #input the configuration {z_{2n+1}}, outputs |\Psi_{z_{2n+1}} 
    Lover2=length(zconfig)
    Ψz=1
    for j=1:Lover2
        zconfig[j]== 0 ? Ψz=kron(z0,Ψz) : Ψz=kron(z1,Ψz)
        jp1=rem(j,Lover2)+1
        #Fixing x-spin on the even sites according to z
        zconfig[j]==zconfig[jp1] ? Ψz=kron(xp,Ψz) : Ψz=kron(xm,Ψz)
    
    end    
    return Ψz
end

#specify the ${z_2n+1}$ configurations first. For L sites, there are 2^{L/2} configurations

#Ψzstates=zeros(dim,2^(div(L,2)))
ρclus=zeros(dim,dim) 
for code=0:2^(div(L,2))-1
    zconfig=codetobasis(code,div(L,2))
    Ψz=getΨzstate(zconfig)
    #Ψzstates[:,code+1]=getΨzstate(zconfig)
    ρclus+=Ψz*Ψz'
end

#vectorize the density matrix
ρclusvec=reshape(copy(ρclus),dim^2); 
normalize!(ρclusvec); 

#density matrix
ρclus=ρclus/tr(ρclus); 


In [189]:
#check the symmetry
norm(Z2ee*ρclusvec-ρclusvec),norm(Z2eI*ρclusvec-ρclusvec),norm(Z2Ie*ρclusvec-ρclusvec),norm(Z2oo*ρclusvec-ρclusvec)

(0.0, 0.0, 0.0, 0.0)

### $\mathcal{L}_0$

$L^{\mu=1}_j= |1\rangle \langle 7 |$, $L^{\mu=2}_j= |2\rangle \langle 8 |$, $L^{\mu=3}_j= |3\rangle \langle 5 |$, $L^{\mu=4}_j= |4\rangle \langle 6 |$. 
$j$ is on the even sites

In [190]:
#local eigenstates and local jump operators for Linb0
#Note the order of kron in Julia is reversed from my convention
st1=kron(z0,kron(xp,z0))  #|1⟩=|0+0⟩
st2=kron(z1,kron(xp,z1))  #|2⟩=|1+1⟩
st3=kron(z1,kron(xm,z0))  #|3⟩=|0-1⟩
st4=kron(z0,kron(xm,z1))  #|4⟩=|1-0⟩
st5=kron(z0,kron(xm,z0))  #|5⟩=|0-0⟩
st6=kron(z1,kron(xm,z1))  #|6⟩=|1-1⟩
st7=kron(z1,kron(xp,z0))  #|7⟩=|0+1⟩
st8=kron(z0,kron(xp,z1))  #|8⟩=|1+0⟩

#Jump operators for Linb0
#L0μ=[(st1)*(st7)',(st2)*(st8)',(st3)*(st5)',(st4)*(st6)']; #strong x strong symmetry
L0μ=[(st1)*(st7+st8)',(st2)*(st7-st8)',(st3)*(st5+st6)',(st4)*(st5-st6)']; #strong x weak symmetry


In [191]:
Linb0=zeros(Linbdim,Linbdim)
for ind=1:Linbdim
    for μ=1:4
        for j=2:2:L #on even sites
            jm1=rem(j+L-2,L)+1
            jp1=rem(j,L)+1
            sites=[jm1,j,jp1]
            Linb0[:,ind]+=Linbcode(L0μ[μ],sites,ind-1,L,dim)
        
        end
    end
end

In [192]:
#checking if ρclusvec is a steady state
@show norm(Linb0*ρclusvec);

norm(Linb0 * ρclusvec) = 9.813077866773594e-17


In [193]:
eigL0=eigen(Linb0);

In [194]:
eigL0.values[end-20:end]

21-element Vector{ComplexF64}:
     -0.9999999999999977 + 0.0im
     -0.9999999999999977 + 0.0im
     -0.9999999999999972 + 0.0im
 -1.7000627803346464e-15 + 0.0im
  -1.375645201390133e-15 + 0.0im
  -6.954811643861321e-16 + 0.0im
   -5.91437117801615e-16 + 0.0im
   -5.81945053524526e-16 - 2.873739942766655e-16im
   -5.81945053524526e-16 + 2.873739942766655e-16im
  -3.852054692752242e-16 + 0.0im
  -2.906780634795668e-16 + 0.0im
  -2.220446049250313e-16 + 0.0im
   -7.47712144757498e-17 + 0.0im
                     0.0 + 0.0im
  1.7555510870879602e-17 - 3.7023869999958335e-16im
  1.7555510870879602e-17 + 3.7023869999958335e-16im
   2.980406711657805e-16 + 0.0im
   3.514975221738889e-16 + 0.0im
   4.449673581118517e-16 + 0.0im
   5.102770061206795e-16 + 0.0im
  1.2629922077467538e-15 + 0.0im

### $\mathcal{L}_1$: decoheres the states 
$L_j = Z_j$ on the odd sites

In [195]:
Linb1=zeros(Linbdim,Linbdim)
for ind=1:Linbdim
        for j=1:2:L
            sites=[j]
            Linb1[:,ind]+=Linbcode(σz,sites,ind-1,L,dim)
        end

end

In [196]:
#Testing if ρclusvec is also a steady state of L0+L1
@show norm((Linb0+Linb1)*ρclusvec);

norm((Linb0 + Linb1) * ρclusvec) = 9.813077866773594e-17


### $\mathcal{L}_2$: adjusts the "population" distribution

$L^{\mu=1}_j = Z_{j-1}\sigma^+_j Z_{j+1}$, $L^{\mu=2}_j = Z_{j-1}\sigma^-_j Z_{j+1}$ on the odd sites

In [197]:
σp=[0 1;0 0]
σm=[0 0;1 0]
L2p=kron(kron(σz,σp),σz)
L2m=kron(kron(σz,σm),σz)
L2μ=[L2p,L2m];

In [198]:
Linb2=zeros(Linbdim,Linbdim)
for ind=1:Linbdim
    for μ=1:2
        for j=1:2:L
            jm1=rem(j+L-2,L)+1
            jp1=rem(j,L)+1
            sites=[jm1,j,jp1]
            Linb2[:,ind]+=Linbcode(L2μ[μ],sites,ind-1,L,dim)
        
        end
    end
end

In [199]:
Linb=Linb0+Linb1+Linb2;

In [200]:
#Testing if ρclusvec is also a steady state of L0+L1+L2
@show norm(Linb*ρclusvec);

norm(Linb * ρclusvec) = 3.589494578342556e-16


In [201]:
eigL=eigen(Linb);

In [202]:
eigL.values[end-2:end]

3-element Vector{ComplexF64}:
      -0.999999999999997 + 0.0im
 -1.2307917031491435e-15 + 0.0im
  -6.052278749947602e-16 + 0.0im

#### Check the symmetries of the steady state

In [203]:
ρst1=eigL.vectors[:,end]
ρst1=ρst1/norm(ρst1)

ρst2=eigL.vectors[:,end-1]
ρst2=ρst2/norm(ρst2)

norm(Z2ee*ρst1-ρst1),norm(Z2eI*ρst1-ρst1),norm(Z2Ie*ρst1+ρst1),norm(Z2oo*ρst1-ρst1)


(1.4801783679127622e-15, 1.0663999071224517, 1.6919784981166948, 8.922936293941372e-16)

In [204]:
norm(Z2ee*ρst2-ρst2),norm(Z2eI*ρst2+ρst2),norm(Z2Ie*ρst2-ρst2),norm(Z2oo*ρst2-ρst2)

(1.36768393675572e-15, 0.20711914538765916, 1.9892465054924409, 1.0228114088382776e-15)

In [205]:
ρstp=(ρst1 + Z2eI*ρst1)/sqrt(2); @show norm(ρstp)
ρstp=ρstp/norm(ρstp)
norm(Z2ee*ρstp-ρstp),norm(Z2eI*ρstp-ρstp),norm(Z2Ie*ρstp-ρstp),norm(Z2oo*ρstp-ρstp)

norm(ρstp) = 1.1964094696401453


(1.217198171731104e-15, 0.0, 1.217198171731104e-15, 6.560335962597594e-16)

In [206]:
ρstm=(ρst1 - Z2eI*ρst1)/sqrt(2); @show norm(ρstm)
ρstm=ρstm/norm(ρstm)
norm(Z2ee*ρstm-ρstm),norm(Z2eI*ρstm+ρstm),norm(Z2Ie*ρstm+ρstm),norm(Z2oo*ρstm-ρstm)

norm(ρstm) = 0.7540586057829899


(2.104987576023561e-15, 0.0, 2.104987576023561e-15, 1.2853279154961122e-15)

In [213]:
ρstpm=(ρst1 + Z2eI*ρst1 - Z2Ie*ρst1 - Z2ee*ρst1)/sqrt(2); @show norm(ρstpm)
ρstpm=ρstpm/norm(ρstpm)
norm(Z2ee*ρstpm-ρstpm),norm(Z2eI*ρstpm+ρstpm),norm(Z2Ie*ρstpm+ρstpm),norm(Z2oo*ρstpm-ρstpm)

norm(ρstpm) = 1.3995294012910787e-15


(1.9996492922815474, 1.9998920669359996, 0.0349047734000307, 0.6568211578073426)

In [207]:
#Checking the overlap between ++ state and the |ρcluster>
dot(ρstp,ρclusvec)

0.9999999999999998 + 0.0im

Understand the wavefunction of $ρ_{--}$. Operate Hadamard gates on the even sites for easier representation


In [208]:
Hadamard=[1 1;1 -1]/sqrt(2);
Id=Matrix(I,2,2)
UHeven=1
for j=1:L
   rem(j,2)==0 ? UHeven=kron(Hadamard,UHeven) : UHeven=kron(Id,UHeven)
end
UHeven2=kron(transpose(UHeven),UHeven);

In [209]:
Hρstm=UHeven2*ρstm;

In [210]:
for i=1:length(ρstm)
    abs(Hρstm[i]) > 1E-10 ? println(Hρstm[i]," ",codetobasis(i-1,2L)[1:L]) : nothing
end

0.35355339059327373 + 0.0im [0, 1, 0, 0]
0.35355339059327373 + 0.0im [1, 1, 0, 0]
0.3535533905932737 + 0.0im [0, 1, 1, 0]
0.3535533905932733 + 0.0im [1, 1, 1, 0]
0.35355339059327373 + 0.0im [0, 0, 0, 1]
0.3535533905932738 + 0.0im [1, 0, 0, 1]
0.35355339059327373 + 0.0im [0, 0, 1, 1]
0.35355339059327334 + 0.0im [1, 0, 1, 1]


In [211]:
norm(Linb0*ρstm)

2.1402518545605134e-15

There are two steady states. It has to be at least two steady states since one of the $Z_2$ symmetry is strong?
Though indeed *ρclus* is in the steady state space.

## Open boundary condition

In [263]:
L=6
dim=2^L
Linbdim=dim^2


4096

In [264]:
#Symmetry operators
U2e=getZ2e(L)
U2o=getZ2o(L);
Id=Matrix(I,dim,dim)
Z2ee=kron(U2e,U2e)
Z2oo=kron(U2o,U2o);
Z2Ie=kron(U2e,Id)
Z2eI=kron(Id,U2e);

In [265]:
#local eigenstates and local jump operators for Linb0
xp=[1,1]/sqrt(2)    #|+⟩
xm=[1,-1]/sqrt(2)   #|-⟩
z0=[1,0]            #|0⟩
z1=[0,-1]           #|1⟩
#Note the order of kron in Julia is reversed from my convention
st1=kron(z0,kron(xp,z0))  #|1⟩=|0+0⟩
st2=kron(z1,kron(xp,z1))  #|2⟩=|1+1⟩
st3=kron(z1,kron(xm,z0))  #|3⟩=|0-1⟩
st4=kron(z0,kron(xm,z1))  #|4⟩=|1-0⟩
st5=kron(z0,kron(xm,z0))  #|5⟩=|0-0⟩
st6=kron(z1,kron(xm,z1))  #|6⟩=|1-1⟩
st7=kron(z1,kron(xp,z0))  #|7⟩=|0+1⟩
st8=kron(z0,kron(xp,z1))  #|8⟩=|1+0⟩

#Jump operators for Linb0
#L0μ=[(st1)*(st7)',(st2)*(st8)',(st3)*(st5)',(st4)*(st6)']; #strong x strong symmetry
L0μ=[(st1)*(st7+st8)',(st2)*(st7-st8)',(st3)*(st5+st6)',(st4)*(st5-st6)']; #strong x weak symmetry


In [266]:
Linb0=zeros(Linbdim,Linbdim)
for ind=1:Linbdim
    for μ=1:4
        for j=2:2:L-1 #on even sites
            jm1=rem(j+L-2,L)+1
            jp1=rem(j,L)+1
            sites=[jm1,j,jp1]
            Linb0[:,ind]+=Linbcode(L0μ[μ],sites,ind-1,L,dim)
        
        end
    end
end

In [267]:
Linb1=zeros(Linbdim,Linbdim)
for ind=1:Linbdim
        for j=1:2:L
            sites=[j]
            Linb1[:,ind]+=Linbcode(σz,sites,ind-1,L,dim)
        end

end

In [268]:
σp=[0 1;0 0]
σm=[0 0;1 0]
L2p=kron(kron(σz,σp),σz)
L2m=kron(kron(σz,σm),σz)
L2μ=[L2p,L2m];

Linb2=zeros(Linbdim,Linbdim)
for ind=1:Linbdim
    for μ=1:2
        for j=1:2:L
            jm1=rem(j+L-2,L)+1
            jp1=rem(j,L)+1
            sites=[jm1,j,jp1]
            Linb2[:,ind]+=Linbcode(L2μ[μ],sites,ind-1,L,dim)
        
        end
    end
end

In [269]:
Linb=Linb0+Linb1+Linb2;

In [270]:
eigL=eigen(Linb);

In [271]:
eigL.values[end-10:end]

11-element Vector{ComplexF64}:
     -1.0000000000000007 + 0.0im
     -0.9999999999999986 + 0.0im
     -0.9999999999999961 + 0.0im
     -0.9999999999999944 + 0.0im
     -0.9999999999999942 + 0.0im
     -0.9999999999999896 + 0.0im
     -0.9999999999999882 + 0.0im
 -1.4469689368691575e-15 + 0.0im
   2.220446049250313e-16 + 0.0im
  2.6520949506316732e-15 + 0.0im
   3.552713678800501e-15 + 0.0im

In [272]:
Id=Matrix(I,dim,dim)
Ivec=reshape(Id,dim^2);

In [273]:
ρst1=eigL.vectors[:,end];
ρst1=ρst1/norm(ρst1)
norm(Z2ee*ρst1-ρst1),norm(Z2eI*ρst1-ρst1),norm(Z2Ie*ρst1+ρst1),norm(Z2oo*ρst1-ρst1)

(1.4142135623730931, 1.4142135623730934, 1.4142135623730931, 1.6290652945229635e-15)

In [274]:
ρst2=eigL.vectors[:,end-1];
ρst2=ρst2/norm(ρst2)
norm(Z2ee*ρst1-ρst1),norm(Z2eI*ρst1-ρst1),norm(Z2Ie*ρst1+ρst1),norm(Z2oo*ρst1-ρst1)

(1.4142135623730931, 1.4142135623730934, 1.4142135623730931, 1.6290652945229635e-15)

In [275]:
ρstpp=(ρst1 + Z2eI*ρst1 + Z2Ie*ρst1+ Z2ee*ρst1); @show norm(ρstpp)
ρstpp=ρstpp/norm(ρstpp)
@show norm(Linb*ρstpp),dot(Ivec,ρstpp)
norm(Z2ee*ρstpp-ρstpp),norm(Z2eI*ρstpp-ρstpp),norm(Z2Ie*ρstpp-ρstpp),norm(Z2oo*ρstpp-ρstpp)

norm(ρstpp) = 2.000000000000006
(norm(Linb * ρstpp), dot(Ivec, ρstpp)) = (6.015048664603527e-15, 2.8284271247461845 + 0.0im)


(0.0, 0.0, 0.0, 1.62906529452296e-15)

In [276]:
ρstpm=(ρst1 + Z2eI*ρst1 - Z2Ie*ρst1- Z2ee*ρst1); @show norm(ρstpm)
ρstpm=ρstpm/norm(ρstpm)
@show norm(Linb*ρstpm),dot(Ivec,ρstpm)
norm(Z2ee*ρstpm+ρstpm),norm(Z2eI*ρstpm-ρstpm),norm(Z2Ie*ρstpm+ρstpm),norm(Z2oo*ρstpm-ρstpm)

norm(ρstpm) = 2.000000000000006
(norm(Linb * ρstpm), dot(Ivec, ρstpm)) = (6.015048664603527e-15, -5.065392549852277e-16 + 0.0im)


(0.0, 0.0, 0.0, 1.62906529452296e-15)

In [277]:
ρstmp=(ρst1 - Z2eI*ρst1 + Z2Ie*ρst1- Z2ee*ρst1); @show norm(ρstmp)
ρstmp=ρstmp/norm(ρstmp)
@show norm(Linb*ρstmp),dot(Ivec,ρstmp)
norm(Z2ee*ρstmp+ρstmp),norm(Z2eI*ρstmp+ρstmp),norm(Z2Ie*ρstmp-ρstmp),norm(Z2oo*ρstmp-ρstmp)

norm(ρstmp) = 2.000000000000006
(norm(Linb * ρstmp), dot(Ivec, ρstmp)) = (6.015048664603527e-15, 5.065392549852277e-16 + 0.0im)


(0.0, 0.0, 0.0, 1.62906529452296e-15)

In [278]:
ρstmm=(ρst1 - Z2eI*ρst1 - Z2Ie*ρst1 + Z2ee*ρst1); @show norm(ρstmm)
ρstmm=ρstmm/norm(ρstmm)
@show norm(Linb*ρstmm),dot(Ivec,ρstmm)
norm(Z2ee*ρstmm-ρstmm),norm(Z2eI*ρstmm+ρstmm),norm(Z2Ie*ρstmm+ρstmm),norm(Z2oo*ρstmm-ρstmm)


norm(ρstmm) = 2.000000000000006
(norm(Linb * ρstmm), dot(Ivec, ρstmm)) = (6.015048664603527e-15, -2.8284271247461845 + 0.0im)


(0.0, 0.0, 0.0, 1.62906529452296e-15)

Understand the wavefunctions of the steady states in the open boundary condition

In [283]:
Hadamard=[1 1;1 -1]/sqrt(2);
Id=Matrix(I,2,2)
UHeven=1
for j=1:L
   rem(j,2)==0 ? UHeven=kron(Hadamard,UHeven) : UHeven=kron(Id,UHeven)
end
UHeven2=kron(transpose(UHeven),UHeven);

In [284]:
Hρstpp=UHeven2*ρstpp;
for i=1:length(ρstpp)
    abs(Hρstpp[i]) > 1E-10 ? println(Hρstpp[i]," ",codetobasis(i-1,2L)) : nothing
end

0.35355339059327295 + 0.0im [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.3535533905932727 + 0.0im [0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0]
0.3535533905932727 + 0.0im [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
0.35355339059327273 + 0.0im [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
0.3535533905932726 + 0.0im [1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1]
0.35355339059327234 + 0.0im [1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1]
0.35355339059327257 + 0.0im [0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1]
0.3535533905932726 + 0.0im [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1]


In [285]:
Hρstmm=UHeven2*ρstmm;
for i=1:length(ρstmm)
    abs(Hρstmm[i]) > 1E-10 ? println(Hρstmm[i]," ",codetobasis(i-1,2L)) : nothing
end

-0.3535533905932726 + 0.0im [1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]
-0.35355339059327234 + 0.0im [1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0]
-0.35355339059327257 + 0.0im [0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0]
-0.3535533905932726 + 0.0im [0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0]
-0.35355339059327295 + 0.0im [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
-0.3535533905932727 + 0.0im [0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1]
-0.3535533905932727 + 0.0im [1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1]
-0.35355339059327273 + 0.0im [1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1]


In [286]:
Hρstpm=UHeven2*ρstpm;
for i=1:length(ρstpm)
    abs(Hρstpm[i]) > 1E-10 ? println(Hρstpm[i]," ",codetobasis(i-1,2L)) : nothing
end

-0.3535533905932726 + 0.0im [1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0]
-0.35355339059327234 + 0.0im [1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0]
-0.35355339059327257 + 0.0im [0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0]
-0.3535533905932726 + 0.0im [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0]
-0.35355339059327295 + 0.0im [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
-0.3535533905932727 + 0.0im [0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1]
-0.3535533905932727 + 0.0im [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1]
-0.35355339059327273 + 0.0im [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1]


In [287]:
Hρstmp=UHeven2*ρstmp;
for i=1:length(ρstmp)
    abs(Hρstmp[i]) > 1E-10 ? println(Hρstmp[i]," ",codetobasis(i-1,2L)) : nothing
end


0.35355339059327295 + 0.0im [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
0.3535533905932727 + 0.0im [0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0]
0.3535533905932727 + 0.0im [1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0]
0.35355339059327273 + 0.0im [1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0]
0.3535533905932726 + 0.0im [1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]
0.35355339059327234 + 0.0im [1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1]
0.35355339059327257 + 0.0im [0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1]
0.3535533905932726 + 0.0im [0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1]
